# [Housing Prices Competition for Kaggle Learn Users](https://www.kaggle.com/c/home-data-for-ml-course)

## Install necessary packages

In [1]:
# !conda install numpy pandas matplotlib seaborn missingno scikit-learn pandas_profiling -y

In [2]:
# !conda install -c conda-forge xgboost -y

## Import necessary packages

In [3]:
import numpy as np
import pandas as pd

from pandas.api.types import CategoricalDtype

import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
from pandas_profiling import ProfileReport

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor

## Read in data

In [4]:
train_file_path = "./../../data/hpc_train.csv"
test_file_path = "./../../data/hpc_test.csv"

In [5]:
pd.options.display.max_columns = None

In [6]:
train = pd.read_csv(train_file_path, 
                    index_col='Id')
train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1999,2000,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,0,953,953,GasA,Ex,Y,SBrkr,953,694,0,1647,0,0,2,1,3,1,TA,7,Typ,1,TA,Attchd,1999.0,RFn,2,460,TA,TA,Y,0,40,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,6,1978,1988,Gable,CompShg,Plywood,Plywood,Stone,119.0,TA,TA,CBlock,Gd,TA,No,ALQ,790,Rec,163,589,1542,GasA,TA,Y,SBrkr,2073,0,0,2073,1,0,2,0,3,1,TA,7,Min1,2,TA,Attchd,1978.0,Unf,2,500,TA,TA,Y,349,0,0,0,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,9,1941,2006,Gable,CompShg,CemntBd,CmentBd,None,0.0,Ex,Gd,Stone,TA,Gd,No,GLQ,275,Unf,0,877,1152,GasA,Ex,Y,SBrkr,1188,1152,0,2340,0,0,2,0,4,1,Gd,9,Typ,2,Gd,Attchd,1941.0,RFn,1,252,TA,TA,Y,0,60,0,0,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500


In [7]:
test = pd.read_csv(test_file_path, 
                   index_col='Id')

In [8]:
train.MSSubClass = train.MSSubClass.astype('category')

train.OverallQual = train.OverallQual.astype(CategoricalDtype(categories=list(range(1, 11)),
                                                              ordered=True))

train.OverallCond = train.OverallCond.astype(CategoricalDtype(categories=list(range(1, 11)), 
                                                              ordered=True))

In [9]:
train.YearBuilt = pd.to_datetime(train.YearBuilt, format='%Y')

train.YearRemodAdd = pd.to_datetime(train.YearRemodAdd, format='%Y')

train.GarageYrBlt = pd.to_datetime(train.GarageYrBlt, format='%Y')

train.MoSold = pd.to_datetime(train.MoSold, format='%M')
train.YrSold = pd.to_datetime(train.YrSold, format='%Y')

In [10]:
yes_no_map = {'Y': True, 'N': False}

train.CentralAir = train.CentralAir.map(yes_no_map).astype('bool')

## Exploratory Data Analysis


### Missing values

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   MSSubClass     1460 non-null   category      
 1   MSZoning       1460 non-null   object        
 2   LotFrontage    1201 non-null   float64       
 3   LotArea        1460 non-null   int64         
 4   Street         1460 non-null   object        
 5   Alley          91 non-null     object        
 6   LotShape       1460 non-null   object        
 7   LandContour    1460 non-null   object        
 8   Utilities      1460 non-null   object        
 9   LotConfig      1460 non-null   object        
 10  LandSlope      1460 non-null   object        
 11  Neighborhood   1460 non-null   object        
 12  Condition1     1460 non-null   object        
 13  Condition2     1460 non-null   object        
 14  BldgType       1460 non-null   object        
 15  HouseStyle     1460 n

In [12]:
cat_col_names_with_na = []
cat_col_names_without_na = []
for col_name in train.select_dtypes(include=['object']):
    print(f"Name: {col_name}\tdtype: {train[col_name].dtype}")
    print(f"nunique: {train[col_name].nunique()}")
    missing_count = train[col_name].isnull().sum()
    count = train[col_name].count()
    if missing_count:
        cat_col_names_with_na.append(col_name)
        print(f"Missing: {missing_count} / {count} -> {missing_count/(missing_count + count)*100}% ")    
    else:
        cat_col_names_without_na.append(col_name)
    display(train[col_name].value_counts(sort=False))
    
    print('-' * 30, '\n')

Name: MSZoning	dtype: object
nunique: 5


FV           65
C (all)      10
RH           16
RM          218
RL         1151
Name: MSZoning, dtype: int64

------------------------------ 

Name: Street	dtype: object
nunique: 2


Pave    1454
Grvl       6
Name: Street, dtype: int64

------------------------------ 

Name: Alley	dtype: object
nunique: 2
Missing: 1369 / 91 -> 93.76712328767123% 


...

------------------------------ 

Name: SaleType	dtype: object
nunique: 9


Con         2
ConLI       5
ConLD       9
ConLw       5
CWD         4
COD        43
Oth         3
WD       1267
New       122
Name: SaleType, dtype: int64

------------------------------ 

Name: SaleCondition	dtype: object
nunique: 6


AdjLand       4
Abnorml     101
Partial     125
Alloca       12
Normal     1198
Family       20
Name: SaleCondition, dtype: int64

------------------------------ 



In [13]:
cat_col_names_without_na

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'Heating',
 'HeatingQC',
 'KitchenQual',
 'Functional',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   MSSubClass     1460 non-null   category      
 1   MSZoning       1460 non-null   object        
 2   LotFrontage    1201 non-null   float64       
 3   LotArea        1460 non-null   int64         
 4   Street         1460 non-null   object        
 5   Alley          91 non-null     object        
 6   LotShape       1460 non-null   object        
 7   LandContour    1460 non-null   object        
 8   Utilities      1460 non-null   object        
 9   LotConfig      1460 non-null   object        
 10  LandSlope      1460 non-null   object        
 11  Neighborhood   1460 non-null   object        
 12  Condition1     1460 non-null   object        
 13  Condition2     1460 non-null   object        
 14  BldgType       1460 non-null   object        
 15  HouseStyle     1460 n

In [15]:
cat_col_names_with_na

['Alley',
 'MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']

In [16]:
train.Alley.fillna('NA', inplace=True)
train.BsmtQual.fillna('NA', inplace=True)
train.BsmtCond.fillna('NA', inplace=True)
train.BsmtExposure.fillna('NA', inplace=True)
train.BsmtFinType1.fillna('NA', inplace=True)
train.BsmtFinType2.fillna('NA', inplace=True)
train.FireplaceQu.fillna('NA', inplace=True)
train.GarageType.fillna('NA', inplace=True)
train.GarageFinish.fillna('NA', inplace=True)
train.GarageQual.fillna('NA', inplace=True)
train.GarageCond.fillna('NA', inplace=True)
train.PoolQC.fillna('NA', inplace=True)
train.Fence.fillna('NA', inplace=True)
train.MiscFeature.fillna('NA', inplace=True)

In [17]:
train.MasVnrType.fillna(train.MasVnrType.mode()[0], inplace=True)

train.Electrical.fillna(train.Electrical.mode()[0], inplace=True)

### Data types

#### object -> categorical

In [34]:
train.MSZoning = train.MSZoning.astype(CategoricalDtype(categories=['A', 'C', 
                                                                    'FV', 'I',
                                                                    'RH','RL', 
                                                                    'RP', 'RM'], 
                                                        ordered=False))

train.MSSubClass = train.MSSubClass.astype(CategoricalDtype(categories=[20, 30, 
                                                                        40, 45, 
                                                                        50, 60, 
                                                                        70, 75, 
                                                                        80, 85, 
                                                                        90, 120, 
                                                                        150, 160,  
                                                                        180, 190], 
                                                            ordered=False))

train.Street = train.Street.astype("category")

train.Alley = train.Alley.astype("category")

train.LotShape = train.LotShape.astype("category")

train.LandContour = train.LandContour.astype("category")

train.Utilities = train.Utilities.astype(CategoricalDtype(categories=['ELO', 'NoSeWa', 
                                                                      'NoSewr', 'AllPub'], 
                                                          ordered=True))

train.LotConfig = train.LotConfig.astype("category")

train.LandSlope = train.LandSlope.astype("category")

train.Neighborhood = train.Neighborhood.astype("category")

train.Condition1 = train.Condition1.astype("category")

train.Condition2 = train.Condition2.astype("category")

train.BldgType = train.BldgType.astype("category")

train.HouseStyle = train.HouseStyle.astype("category")

train.RoofStyle = train.RoofStyle.astype("category")

train.RoofMatl = train.RoofMatl.astype("category")

Exterior_cat = CategoricalDtype(categories=
                                train.Exterior1st.unique().tolist() + 
                                ['Other', 'PreCast'], 
                                ordered=False)
train.Exterior1st = train.Exterior1st.astype(Exterior_cat)
train.Exterior2nd = train.Exterior2nd.astype(Exterior_cat)

MasVnrType_cat = CategoricalDtype(categories=
                                train.MasVnrType.unique().tolist() + 
                                ['CBlock'], 
                                ordered=False)
train.MasVnrType = train.MasVnrType.astype(MasVnrType_cat)

Exter_cat = CategoricalDtype(categories=['Po', 'Fa', 'TA', 'Gd', 'Ex'], 
                                ordered=True)
train.ExterQual = train.ExterQual.astype(Exter_cat)
train.ExterCond = train.ExterQual.astype(Exter_cat)
train.HeatingQC = train.HeatingQC.astype(Exter_cat)
train.KitchenQual = train.KitchenQual.astype(Exter_cat)


# TODO: what is the best - no or something?
Bsmt_cat = CategoricalDtype(categories=['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'], 
                            ordered=True)
train.BsmtQual = train.BsmtQual.astype(Bsmt_cat)
train.BsmtCond = train.BsmtCond.astype(Bsmt_cat)
train.FireplaceQu = train.FireplaceQu.astype(Bsmt_cat)
train.GarageQual = train.GarageQual.astype(Bsmt_cat)
train.GarageCond = train.GarageCond.astype(Bsmt_cat)
train.PoolQC = train.PoolQC.astype(Bsmt_cat)


train.Foundation = train.Foundation.astype("category")

train.BsmtExposure = train.BsmtExposure.astype("category")

Bsmt_fin = CategoricalDtype(categories=['NA', 'Unf', 'LwQ', 'Rec', 
                                        'BLQ', 'ALQ', 'GLQ'], 
                            ordered=True)
train.BsmtFinType1 = train.BsmtFinType1.astype(Bsmt_fin)
train.BsmtFinType2 = train.BsmtFinType2.astype(Bsmt_fin)


train.Heating = train.Heating.astype("category")

Electrical_cat = CategoricalDtype(categories=['Mix', 'FuseP', 
                                              'FuseF', 'FuseA',
                                              'SBrkr'], 
                                  ordered=True)
train.Electrical = train.Electrical.astype(Electrical_cat)

Functional_cat = CategoricalDtype(categories=['Sal', 'Sev', 
                                              'Maj2', 'Maj1',
                                              'Mod', 'Min2', 'Min1', 'Typ'], 
                                  ordered=True)
train.Functional = train.Functional.astype(Functional_cat)

# ordered?
train.GarageType = train.GarageType.astype("category")

GarageFinish_cat = CategoricalDtype(categories=['NA', 'Unf', 
                                              'RFn', 'Fin'], 
                                  ordered=True)
train.GarageFinish = train.GarageFinish.astype(GarageFinish_cat)

PavedDrive_cat = CategoricalDtype(categories=['N', 'P', 'Y'], 
                                  ordered=True)
train.PavedDrive = train.PavedDrive.astype(PavedDrive_cat)

PavedDrive_cat = CategoricalDtype(categories=['NA', 'MnWw', 
                                              'GdWo', 'MnPrv', 'GdPrv'], 
                                  ordered=True)
train.Fence = train.Fence.astype(PavedDrive_cat)

train.MiscFeature = train.MiscFeature.astype("category")

train.SaleType = train.SaleType.astype("category")

train.SaleCondition = train.SaleCondition.astype("category")


In [35]:
train.select_dtypes(exclude=['category', 'number', 'datetime', 'bool']).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Empty DataFrame